<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Tabular Playground Series - Jul 2021
</h1>
</div>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/25226/logos/header.png?t=2021-01-27-17-34-31)

In this competition you are predicting the values of air pollution measurements over time, based on basic weather information (temperature and humidity) and the input values of 5 sensors.

The three target values to you to predict are: target_carbon_monoxide, target_benzene, and target_nitrogen_oxides

## Data
* train.csv - the training data, including the weather data, sensor data, and values for the 3 targets
* test.csv - the same format as train.csv, but without the target value; your task is to predict the value for each of these targets.
* sample_submission.csv - a sample submission file in the correct format.

### I recently interested in XAI using model's weights, bias, and Activation function.
### So, I try it this TPS - Jul !

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import matplotlib.font_manager as fm
font = fm.FontProperties(fname='../input/staatfont/Staatliches-Regular.ttf')

train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')

### I don't consisder date_time 😀

In [ ]:
train.head(5)

#### After Drop data_time column

In [ ]:
train = train.drop('date_time',axis=1)
train.head()

### Data Processing - Scaler

In [ ]:
X_train = train[['deg_C','relative_humidity','absolute_humidity',
          'sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']]
y_train = train[['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

print('After Data Processing Data Shape : ',X_train.shape, y_train.shape)

### Modeling - DNN

In [ ]:
from keras import layers, models
import tensorflow as tf

model = models.Sequential()
model.add(layers.Dense(6, input_shape=X_train.shape, activation='relu'))
model.add(layers.Dense(3))
model.compile(loss = tf.keras.losses.MeanSquaredLogarithmicError(), optimizer='adam')
model.summary()

In [ ]:
history = model.fit(X_train,y_train,epochs=500, batch_size=256, validation_split=0.3,verbose=0)

In [ ]:
fig = plt.figure(figsize=(13,8))
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],marker='o',label='val_loss' )
plt.legend(fontsize =15)
plt.title('Model fit result',font=font, fontsize=20)
plt.show()

### I don't focus making model.
### So, I just build simple DNN model

In [ ]:
weight = model.get_weights()
layer1 = pd.DataFrame([list(range(1,9,1)),np.ones((8))]).T
layer2 = pd.DataFrame([list(range(2,8,1)),np.ones((6))]).T
layer3 = pd.DataFrame([list(range(4,7,1)),np.ones((3))]).T
layer2[1] = layer2[1]*2
layer3[1] = 3
point = pd.concat([layer1,layer2,layer3]).reset_index(drop=True)
point.columns = ['y', 'x']

print("weight value shape : ",model.get_weights()[0].shape, model.get_weights()[2].shape)
print("bias value shape : ", model.get_weights()[1].shape, model.get_weights()[3].shape)

### We have 4 weights totally
### 2 weights + 2 bias

### Do XAI

In [ ]:
col = train.columns

fig = plt.figure(figsize=(20,13))
ax = fig.add_subplot(111)
plt.scatter( x= point['x'],y=point['y'], s=400,color='black')

weight_i = 0
weight_j =0
for i in range(1,9):
    weight_j =0
    for j in range(2,8):
        tmp = pd.DataFrame({'x':[1,2], 'y': [i,j]})
        we = weight[0][weight_i][weight_j]
        bias = weight[1][weight_j]/8
        value = we + bias
        if value >= 0:
            color = 'green'
        else:
            color = 'red'
        sns.lineplot(x=tmp['x'],y=tmp['y'],ci=None,lw=np.abs(we),color=color)
        weight_j = weight_j+1
    weight_i = weight_i+1
    
weight_i = 0
weight_j =0
for i in range(2,8):
    weight_j =0
    for j in range(4,7):
        tmp = pd.DataFrame({'x':[2,3], 'y': [i,j]})
        we = weight[2][weight_i][weight_j]
        bias = weight[3][weight_j]/6
        value = we + bias
        if value >= 0:
            color = 'green'
        else:
            color = 'red'
        sns.lineplot(x=tmp['x'],y=tmp['y'],lw=np.abs(we),ci=None,color=color)
        weight_j = weight_j+1
    weight_i = weight_i+1
        
plt.axis('off')
plt.text(s='Input_layer', x=1,y=9,font = font, fontsize=25,va='center',ha='center')
plt.text(s='hidden_layer', x=2,y=9,font = font,fontsize=25,va='center',ha='center')
plt.text(s='out_layer', x=3,y=9,font = font,fontsize=25,va='center',ha='center')
plt.text(s='DNN - Not consider Activation function', x=0.9,y=10,va='center',ha='left',font = font,fontsize=50)

for i in range(len(X_train[0])):
    s = col[i]
    plt.text(s = s, x =0.9, y = i+1,font=font, fontsize=18,va='center',ha='right')
for i in range(len(col[8:])):
    s=col[i+8]
    plt.text(s = s, x =3.1, y = i+4,font=font, fontsize=18,va='center',ha='left')
    
plt.text(s='Consider Relu', x=2,y=8,font = font,fontsize=18,va='center',ha='center')

ax.axvspan(1.8, 2.2, fc="gray", alpha=0.2)

plt.show()

### This is totally, DNN model's visualization using weights, and bias
### Not consider activation function like relu

In [ ]:
col = train.columns
index = 0

fig = plt.figure(figsize=(20,13))
ax = fig.add_subplot(111)
plt.scatter( x= point['x'],y=point['y'], s=400,color='black')

weight_i = 0
weight_j =0
for i in range(1,9):
    weight_j =0
    for j in range(2,8):
        tmp = pd.DataFrame({'x':[1,2], 'y': [i,j]})
        we = weight[0][weight_i][weight_j]
        bias = weight[1][weight_j]/8
        value = we + bias
        if value >= 0:
            color = 'green'
        else:
            color = 'red'
        sns.lineplot(x=tmp['x'],y=tmp['y'],ci=None,lw=np.abs(we),color=color)
        weight_j = weight_j+1
    weight_i = weight_i+1
    
weight_i = 0
weight_j =0
for i in range(2,8):
    weight_j =0
    for j in range(4,7):
        tmp = pd.DataFrame({'x':[2,3], 'y': [i,j]})
        we = weight[2][weight_i][weight_j]
        bias = weight[3][weight_j]/6
        value = we + bias
        if value >= 0:
            color = 'green'
        else:
            color = 'red'
        sns.lineplot(x=tmp['x'],y=tmp['y'],lw=np.abs(we),ci=None,color=color)
        weight_j = weight_j+1
    weight_i = weight_i+1
        
plt.axis('off')
plt.text(s='Input_layer', x=1,y=9,font = font, fontsize=25,va='center',ha='center')
plt.text(s='hidden_layer', x=2,y=9,font = font,fontsize=25,va='center',ha='center')
plt.text(s='out_layer', x=3,y=9,font = font,fontsize=25,va='center',ha='center')
plt.text(s='Example : X_train[0]', x=0.9,y=10,va='center',ha='left',font = font,fontsize=50)

for i in range(len(col[:8])):
    s = col[i]
    plt.text(s = s, x =0.9, y = i+1,font=font, fontsize=18,va='center',ha='right')
    value = round(X_train[index][i],4)
    plt.text(s =value, x=0.9, y=i+0.7,font=font, fontsize=18,va='center',ha='right')
    
for i in range(len(col[8:])):
    s=col[i+8]
    plt.text(s = s, x =3.1, y = i+4,font=font, fontsize=18,va='center',ha='left')
    
tmp_arr = []
for i in range(0,6):
    value = 0
    for j in range(0,8):
        value = value + X_train[index][j] * weight[0][j][i] + weight[1][i]/8
    tmp_arr.append(value)
    value =round(value,4)
    if value>0:
        plt.text(s = value, x=1.9,y=i+2, font=font, fontsize=18,va='center',ha='center')
        plt.text(s = value, x=2.1,y=i+2, font=font, fontsize=18,va='center',ha='center')
    else:
        tmp_arr[i] = 0
        plt.text(s = value, x=1.9,y=i+2, font=font, fontsize=18,va='center',ha='center')
        plt.text(s = 0, x=2.1,y=i+2, font=font, fontsize=18,va='center',ha='center')    


tmp_arr2 = []
for i in range(0,3):
    value = 0
    for j in range(0,6):
        value = value + tmp_arr[j] * weight[2][j][i] + weight[3][i]/6
    value =round(value,3)
    tmp_arr2.append(value)
    plt.text(s = value, x =3.2, y = i+3.8, font=font, fontsize=18,va='center',ha='center')
plt.text(s='Consider Relu', x=2,y=8,font = font,fontsize=18,va='center',ha='center')
ax.axvspan(1.8, 2.2, fc="gray", alpha=0.2)

plt.show()

### Look example : X_train[0] 

### It's result same with model's result

In [ ]:
print('keras model predict : ', model.predict(X_train[0:1]))

## Then, What columns impact to the result ?

In [ ]:
zero_value = []
for i in range(len(tmp_arr)):
    if tmp_arr[i] == 0:
        zero_value.append(i)

impact = []
tmp_ar = []
for i in range(0,8):
    columns_value = []
    for j in range(0,6):
        value = X_train[index][i] * weight[0][i][j] + (weight[1][j]/8)
        columns_value.append(value)
        
    for i in zero_value:
        columns_value[i]=0 
    tmp_ar.append(columns_value)
    result = []
    for j in range(0,3):
        value = 0
        for k in range(0,6):
            value = value + columns_value[k] * weight[2][k][j] + weight[3][j]/48
        result.append(value)
    impact.append(result)

# print(np.sum(impact,axis=0))

impact = pd.DataFrame(impact)
impact.index = col[:8]
impact.columns = col[8:]

impact['target_carbon_monoxide'] = impact['target_carbon_monoxide'] / impact['target_carbon_monoxide'].max()
impact['target_benzene'] = impact['target_benzene'] / impact['target_benzene'].max()
impact['target_nitrogen_oxides'] = impact['target_nitrogen_oxides'] / impact['target_nitrogen_oxides'].max()

In [ ]:
col = train.columns
index = 0

fig = plt.figure(figsize=(30,13))
spec = gridspec.GridSpec(ncols=7, nrows=3, figure=fig)
ax = fig.add_subplot(spec[:,:4])
plt.scatter( x= point['x'],y=point['y'], s=400,color='black')

weight_i = 0
weight_j =0
for i in range(1,9):
    weight_j =0
    for j in range(2,8):
        tmp = pd.DataFrame({'x':[1,2], 'y': [i,j]})
        we = weight[0][weight_i][weight_j]
        bias = weight[1][weight_j]/8
        value = we + bias
        if value >= 0:
            color = 'green'
        else:
            color = 'red'
        sns.lineplot(x=tmp['x'],y=tmp['y'],ci=None,lw=np.abs(we),color=color)
        weight_j = weight_j+1
    weight_i = weight_i+1
    
weight_i = 0
weight_j =0
for i in range(2,8):
    weight_j =0
    for j in range(4,7):
        tmp = pd.DataFrame({'x':[2,3], 'y': [i,j]})
        we = weight[2][weight_i][weight_j]
        bias = weight[3][weight_j]/6
        value = we + bias
        if value >= 0:
            color = 'green'
        else:
            color = 'red'
        sns.lineplot(x=tmp['x'],y=tmp['y'],lw=np.abs(we),ci=None,color=color)
        weight_j = weight_j+1
    weight_i = weight_i+1
        
plt.axis('off')
plt.text(s='Input_layer', x=1,y=9,font = font, fontsize=25,va='center',ha='center')
plt.text(s='hidden_layer', x=2,y=9,font = font,fontsize=25,va='center',ha='center')
plt.text(s='out_layer', x=3,y=9,font = font,fontsize=25,va='center',ha='center')
plt.text(s='Example : X_train[0]', x=0.9,y=10,va='center',ha='left',font = font,fontsize=50)

for i in range(len(col[:8])):
    s = col[i]
    plt.text(s = s, x =0.9, y = i+1,font=font, fontsize=18,va='center',ha='right')
    value = round(X_train[index][i],4)
    plt.text(s =value, x=0.9, y=i+0.7,font=font, fontsize=18,va='center',ha='right')
    
for i in range(len(col[8:])):
    s=col[i+8]
    if i == 0:
        plt.text(s = s, x =3.1, y = i+4,font=font, fontsize=18,va='center',ha='left',color='b')
    elif i == 1:
        plt.text(s = s, x =3.1, y = i+4,font=font, fontsize=18,va='center',ha='left',color='y')
    else:
        plt.text(s = s, x =3.1, y = i+4,font=font, fontsize=18,va='center',ha='left',color='r')

    
tmp_arr = []
for i in range(0,6):
    value = 0
    for j in range(0,8):
        value = value + X_train[index][j] * weight[0][j][i] + weight[1][i]/8
    tmp_arr.append(value)
    value =round(value,4)
    if value>0:
        plt.text(s = value, x=1.9,y=i+2, font=font, fontsize=18,va='center',ha='center')
        plt.text(s = value, x=2.1,y=i+2, font=font, fontsize=18,va='center',ha='center')
    else:
        tmp_arr[i] = 0
        plt.text(s = value, x=1.9,y=i+2, font=font, fontsize=18,va='center',ha='center')
        plt.text(s = 0, x=2.1,y=i+2, font=font, fontsize=18,va='center',ha='center')    


tmp_arr2 = []
for i in range(0,3):
    value = 0
    for j in range(0,6):
        value = value + tmp_arr[j] * weight[2][j][i] + weight[3][i]/6
    value =round(value,3)
    tmp_arr2.append(value)
    plt.text(s = value, x =3.2, y = i+3.8, font=font, fontsize=18,va='center',ha='center')
plt.text(s='Consider Relu', x=2,y=8,font = font,fontsize=18,va='center',ha='center')
ax.axvspan(1.8, 2.2, fc="gray", alpha=0.2)

ax2 = fig.add_subplot(spec[2,5:])
plt.barh(width=impact['target_carbon_monoxide'],y=list(range(0,8)))
plt.axis('off')
plt.text(s= 'target_carbon_monoxide',x=0, y=9,va='center',ha='center',font=font,fontsize=15)
for i in range(len(col[:8])):
    s = col[i]
    value = impact.iloc[i][0]
    if value < 0:
        plt.text(s=s,x=value-0.1, y= i,font=font,fontsize=13,va='center',ha='right')
    else:
        plt.text(s=s,x=value+0.1, y= i,font=font,fontsize=13,va='center',ha='left')
plt.xlim(-1,3)

ax3 = fig.add_subplot(spec[1,5:])
plt.barh(width=impact['target_benzene'],y=list(range(0,8)),color ='y')
plt.axis('off')
plt.text(s= 'target_benzene',x=0, y=9,va='center',ha='center',font=font,fontsize=15)
for i in range(len(col[:8])):
    s = col[i]
    value = impact.iloc[i][1]
    if value < 0:
        plt.text(s=s,x=value-0.1, y= i,font=font,fontsize=13,va='center',ha='right')
    else:
        plt.text(s=s,x=value+0.1, y= i,font=font,fontsize=13,va='center',ha='left')
plt.xlim(-1,3)


ax4 = fig.add_subplot(spec[0,5:])
plt.barh(width=impact['target_nitrogen_oxides'],y=list(range(0,8)),color ='r')
plt.axis('off')
plt.text(s= 'target_nitrogen_oxides',x=0, y=9,va='center',ha='center',font=font,fontsize=15)
for i in range(len(col[:8])):
    s = col[i]
    value = impact.iloc[i][2]
    if value < 0:
        plt.text(s=s,x=value-0.1, y= i,font=font,fontsize=13,va='center',ha='right')
    else:
        plt.text(s=s,x=value+0.1, y= i,font=font,fontsize=13,va='center',ha='left')
plt.xlim(-1,3)


plt.show()

### Look example,
### The sensor data's impact is bigger than others humidity, deg_c

### If you have any good idea to visualization Deep learning model,
### Feel free to give me comments! 
### I always welcome your comments ~

### And if you want to view my other work - [LSTM_XAI](https://www.kaggle.com/jeongbinpark/looking-lstm-detailly-trying-xai)
### previous work - [TPS-Jun](https://www.kaggle.com/jeongbinpark/tps-jun-try-visualization-dnn-model2)